# COVID-19's Impact on Healthcare Accessibility
### By: Tristan Call and Maria Elena Aviles-Baquero

## Introductory

## Data Analysis

TODO:
- load in data to use
- create analysis functions (something that takes in specific attributes and returns them)

In [6]:
# load data function: through MyPyTable()

# data analysis functions:
# group_by()
# get_column() - MyPyTable
# plotting - whatever we used in class
    # save_graph() function
# output result files (? TBD)

# summary_stats from MyPyTable

# some useful mysklearn package import statements and reloads
import importlib

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

# uncomment once you paste your mypytable.py into mysklearn package
import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.plot_utils
importlib.reload(mysklearn.plot_utils)
import mysklearn.plot_utils as plot_utils

# import mysklearn.myclassifiers
# importlib.reload(mysklearn.myclassifiers)
# from mysklearn.myclassifiers import MyKNeighborsClassifier, MySimpleLinearRegressor, MyNaiveBayesClassifier

# import mysklearn.myevaluation
# importlib.reload(mysklearn.myevaluation)
# import mysklearn.myevaluation as myevaluation


# STEPS
# 1. Focus on Washington (for now aka. generalize code (as always))
# 2. create default values


import os
import pandas as pd

# loads in the Healthcare Access during the Coronavirus Pandemic into the variable covid
covid_file = os.path.join("input_data", "Indicators_of_Reduced_Access_to_Care_Due_to_the_Coronavirus_Pandemic_During_Last_4_Weeks.csv")
covid = MyPyTable().load_from_file(covid_file)

# Add things to accomplish below:


In [40]:
week21_filename = os.path.join("input_data", "pulse2020_puf_21.sas7bdat")
alldata = pd.read_sas(week21_filename)

relevant_attributes = ["TBIRTH_YEAR", "EGENDER", "RHISPANIC", "RRACE", "EEDUC", "INCOME", "DELAY", "NOTGET"]

data = alldata[["TBIRTH_YEAR", "EGENDER", "RHISPANIC", "RRACE", "EEDUC", "INCOME", "DELAY", "NOTGET"]]
working_data_filename = os.path.join("input_data", "week21_working.csv")
nafree_data = data.dropna()
print(nafree_data)
nafree_data.to_csv(working_data_filename)

       TBIRTH_YEAR  EGENDER  RHISPANIC  RRACE  EEDUC  INCOME  DELAY  NOTGET
1           1969.0      2.0        1.0    1.0    7.0     6.0    1.0     2.0
2           1959.0      2.0        1.0    1.0    7.0     4.0    1.0     1.0
4           1967.0      1.0        1.0    1.0    4.0     6.0    2.0     2.0
5           1965.0      1.0        1.0    1.0    7.0     6.0    2.0     2.0
6           1962.0      2.0        1.0    2.0    4.0     1.0    2.0     2.0
...            ...      ...        ...    ...    ...     ...    ...     ...
69938       1964.0      2.0        1.0    1.0    7.0     8.0    2.0     2.0
69939       1984.0      2.0        1.0    1.0    4.0     5.0    1.0     1.0
69940       1959.0      2.0        1.0    1.0    3.0     6.0    2.0     2.0
69941       1997.0      2.0        1.0    1.0    5.0     3.0    1.0     1.0
69942       1954.0      1.0        1.0    1.0    7.0     6.0    2.0     2.0

[56661 rows x 8 columns]


In [ ]:
table = MyPyTable()
table.load_from_file(working_data_filename)

print(table.data)